# Fitting photometry and spectra together

In [ ]:
from os import path

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib

In [3]:
ppxf_dir = path.dirname(path.realpath(ppxf_package.__file__))

## Read the observed galaxy spectrum

Read SDSS DR8 galaxy spectrum taken from here http://www.sdss3.org/dr8/
The spectrum is *already* log rebinned by the SDSS DR8 pipeline and log_rebin should not be used in this case

In [4]:
file = ppxf_dir + '/spectra/NGC3073_SDSS_DR8.fits'
hdu = fits.open(file)
t = hdu[1].data
z = float(hdu[1].header["Z"]) # SDSS redshift estimate

Only use the wavelength range in common between galaxy and stellar library.

In [7]:
mask = (t['wavelength'] > 3540) & (t['wavelength'] < 7409)
flux = t['flux'][mask]
galaxy = flux/np.median(flux)   # Normalize spectrum to avoid numerical issues
wave = t['wavelength'][mask]

The SDSS wavelengths are in vacuum, while the MILES ones are in air. For a rigorous treatment, the SDSS vacuum wavelengths should be converted into air wavelengths and the spectra should be resampled. To avoid resampling, given that the wavelength dependence of the correction is very weak, I approximate it with a constant factor.


In [8]:
wave *= np.median(util.vac_to_air(wave)/wave)